In [162]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats

# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value. I am going to build an investing strategy that selects the 50 stocks from the sp 500 with the best value metrics into a equal-weight portfolio.

In [173]:
#This csv the symbols of the sp5000.
stocks = pd.read_csv('sp_500_stocks.csv')
#The "IEX_CLOUD_API_TOKEN" is the identification token fot the iex cloud api.
from secrets import IEX_CLOUD_API_TOKEN

As the information from the iex cloud is retrieved in http requests, it is very slow to request information for one stock at the time. Therefore, I created batches of the size of 100 for my requests.

In [174]:
symbol_groups = [','.join(stocks['Ticker'][i:i+100]) for i in range(0, len(stocks.index), 100)]

I will use a `composite` basket of valuation metrics to build robust quantitative value strategies.

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

I use the abbreviation `rv` for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [175]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

In [176]:
for symbol_string in symbol_groups:
    call = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(call).json()

    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda           = data[symbol]['advanced-stats']['EBITDA']
        gross_profit     = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        row = pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ], index = rv_columns)

        rv_dataframe = rv_dataframe.append( row, ignore_index = True)

There is probabliy some missing data which I will have to clean.

In [177]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio',  'EV/EBITDA', 'EV/GP']:
    
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)



Now I calculate value score percentiles for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

In [178]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

In [179]:
for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100
 
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,160.611,N/A,51.25,0.821782,10.09,0.79802,8.1000,0.815842,32.634235,0.885149,14.902325,0.827723,N/A
1,AAL,21.355,N/A,-4.23,0.067327,-1.80,0.051485,0.5691,0.045545,-7.709864,0.027723,1.545720,0.061386,N/A
2,AAP,240.491,N/A,25.00,0.462376,4.60,0.566337,1.4000,0.186139,13.092539,0.409901,3.012720,0.138614,N/A
3,AAPL,159.110,N/A,14.17,0.245545,19.90,0.918812,3.5000,0.49703,10.958115,0.29505,8.532497,0.570297,N/A
4,ABBV,118.930,N/A,31.59,0.609901,16.54,0.89901,3.9100,0.546535,11.479413,0.316832,8.002534,0.546535,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,130.570,N/A,29.77,0.580198,-4.88,0.047525,5.9300,0.733663,21.756609,0.718812,10.244741,0.669307,N/A
501,ZBH,145.630,N/A,33.60,0.635644,2.54,0.333663,4.0600,0.560396,21.377533,0.708911,7.044030,0.429703,N/A
502,ZBRA,612.060,N/A,42.00,0.748515,11.98,0.831683,6.0700,0.745545,31.130698,0.871287,12.984225,0.772277,N/A
503,ZION,66.720,N/A,6.57,0.089109,1.40,0.125743,3.1800,0.455446,5.336827,0.061386,2.840436,0.126733,N/A


The RV Score (which stands for Robust Value) is the value score that we'll use to filter for stocks in this investing strategy. The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

In [180]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []

    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])

    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

Now I filter the 50 best value stocks.

In [181]:
rv_dataframe.sort_values('RV Score', ascending= False, inplace=True)
rv_dataframe.reset_index(drop = True, inplace=True)
rv_dataframe.drop(list(range(50,len(rv_dataframe.index))), inplace=True)

And calculate the number of shares to buy for a portfoilo of 1000000$

In [182]:
position_size = float(1000000) / len(rv_dataframe.index)

for i in rv_dataframe.index:
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,NOW,729.980,27,664.94,0.994059,38.94,0.962376,25.62,0.986139,199.978368,0.99802,33.067432,0.986139,0.985347
1,PAYC,514.950,38,183.00,0.980198,38.99,0.964356,33.16,0.994059,113.458017,0.992079,37.926642,0.994059,0.98495
2,DXCM,639.391,31,119.66,0.962376,29.18,0.950495,26.80,0.988119,130.961823,0.99604,36.612334,0.992079,0.977822
3,NVDA,286.690,69,106.10,0.956436,32.78,0.956436,30.91,0.992079,77.521385,0.978218,47.582562,0.99604,0.975842
4,FTNT,348.160,57,108.30,0.958416,53.85,0.974257,18.54,0.962376,80.078843,0.980198,23.387125,0.956436,0.966337
5,IDXX,656.760,30,78.58,0.938614,73.20,0.982178,17.72,0.952475,57.206129,0.962376,30.322937,0.980198,0.963168
6,ISRG,368.880,54,122.93,0.966337,11.48,0.825743,24.62,0.982178,59.018142,0.964356,31.946403,0.984158,0.944554
7,INTU,631.670,31,87.27,0.952475,17.50,0.905941,18.13,0.956436,60.911360,0.968317,20.475400,0.918812,0.940396
8,TYL,560.057,35,145.69,0.974257,10.65,0.807921,15.87,0.938614,101.984685,0.988119,34.425675,0.988119,0.939406
9,ADBE,696.180,28,58.10,0.857426,21.86,0.930693,20.84,0.976238,54.113781,0.956436,23.022441,0.954455,0.93505


And there you go! A list of the 50 best stocks based of the avarage of 4 metricies.